# Test Notebook for Contact Space Characterization with Behler-Parrinello Descriptors

## 1. Maps Generation from Input

In [ ]:
from mapsy.maps import MapsFromFile

Create maps object

In [ ]:
mos2_defect_bpsf = MapsFromFile('./input-files/bpsf.yaml')

Compute maps on the contact space

In [ ]:
maps = mos2_defect_bpsf.atcontactspace()

The resulting maps contain the position of the points, their probability, and the descriptors values

In [ ]:
maps

We can convert the maps to volumetric data and visualize it

In [ ]:
from mapsy.data import VolumetricField
index: int = 12
volumetric: VolumetricField = mos2_defect_bpsf.tovolumetric(maps.iloc[:,index])
volumetric.plotprojections([4.9,6.2,5.5])

We can select features based on their standard deviation

In [ ]:
features = maps.drop(columns=['x','y','z'])
features = features[features.columns[(features.std()>5.e-1)]]
features

We can compute symmetry functions on arbitrary points by passing the coordinates to the `.atpoints()` method of the maps object

In [ ]:
mos2_defect_bpsf.atpoints([[0.,0.,0.]])

## 2. Maps Generation from Input Components

This reads the input file

In [ ]:
from mapsy.io.input import Input
input: Input = Input('./bpsf.yaml')

Read the system

In [ ]:
from mapsy.io import SystemParser
from mapsy.data import System
mos2_defect: System = SystemParser(input.system).parse()

Read the symmetry functions

In [ ]:
from mapsy.symfunc.parser import SymmetryFunctionsParser
from mapsy.symfunc import SymmetryFunction
symmetryfunctions: list[SymmetryFunction] = SymmetryFunctionsParser(input.symmetryfunctions).parse()

Read the contact space

In [ ]:
from mapsy.io.parser import ContactSpaceGenerator
from mapsy.boundaries import ContactSpace
contactspace: ContactSpace = ContactSpaceGenerator(input.contactspace).generate(mos2_defect)

Create `Maps` object

In [ ]:
from mapsy.maps import Maps
mos2_defect_bpsf: Maps = Maps(mos2_defect,symmetryfunctions,contactspace)

In [ ]:
mos2_defect_bpsf.atpoints([[0.,0.,0.]])

## 3. Manual Generation of Components: System
We can generate the system under study by reading an `xyz+` or a `cube` file. 

Use the `mapsy.io.parser` module to parse the modified xyz file with the atomic positions in units of alat

In [ ]:
from mapsy.io import XYZParser
from mapsy.data import System
system: System = XYZParser('../examples/bp/MoS2_defect.xyz', units='alat').systemparse()

In [ ]:
system.atoms

while for a `cube` file we can use the corresponding parser

In [ ]:
from mapsy.io import CubeParser
from mapsy.data import System
system: System = CubeParser('../examples/cubefiles/fukui_negative_defect.cube').systemparse()

In [ ]:
system.atoms

We can adjust the components that are not set automatically

In [ ]:
system.dimension = 2 # 2D system
system.axis = 2 # 0: X, 1: Y, 2: Z (for 2D system this is the axis perpendicular to the system's plane)

We can create a `Mapsy.system` also from an `Ase.Atom` object and a `Mapsy.Grid`

In [ ]:
from ase import Atoms
from ase.build import mx2
atoms: Atoms = mx2('MoS2', '2H', a = 3.18, size = (2,2,1), vacuum = 15)

In [ ]:
from mapsy.data import Grid
grid: Grid = Grid(cell=atoms.cell)

In [ ]:
from mapsy.data import System
system: System = System(grid, atoms)

In [ ]:
system.atoms

## 4. Manual Generation of Components: Symmetry Functions


We can create symmetry functions using the BP constructor and passing the relevant parameters. 

In [ ]:
from mapsy.symfunc import BPSymmetryFunction
symmetryfunctions: list[BPSymmetryFunction] = []
for order in range(1,4):
    bpsf = BPSymmetryFunction(order = order, radius=5, cutofftype='cos', etas=[0.03, 3.], rss=[0., 0.], lambdas=[1., -1.], kappas=[1, 2, 4, 8])
    symmetryfunctions.append(bpsf)

`System` and `SymmetryFunction` are the only components strictly needed to compute features at arbitrary points. We can define a `Maps` instance from these and use its `atpoints()` method

In [ ]:
system: System = SystemParser(input.system).parse()
mos2_defect_bpsf = Maps(system,symmetryfunctions)
mos2_defect_bpsf.atpoints([[0.,0.,0.]])

## 5. Manual Generation of Components: Contact Space
If we want to compute maps on the contact space, we need to also define this component.

Reload the system to keep it consistent with first test. NOTE: in order for the SystemBoundary to generate an interface with the correct dimensionality (i.e. a 2D planar interface for our 2D material), we need to make sure to add the `system.dimension` and `system.axis` components to our `MapSy.System` instance.

In [ ]:
from mapsy.io import XYZParser
from mapsy.data import System
system: System = XYZParser('../examples/bp/MoS2_defect.xyz', units='alat').systemparse()
system.dimension = 2 # 2D system
system.axis = 2 # 0: X, 1: Y, 2: Z (for 2D system this is the axis perpendicular to the system's plane)

Before creating the boundary and the contact space, we need to decide the resolution of the grid used for the contact space. This follows the same convention as Quantum Espresso, with a cutoff in Ry associated with the kinetic energy of the plane waves used for the Fourier expansion. NOTE: the grid dimensions (`scalars`) are determined according to the system cell. 

In [ ]:
from mapsy.utils import setscalars
cutoff = 10 # reciprocal space cutoff in Ry (same convetion as QE)
scalars = setscalars(system.grid.cell,cutoff)
print(scalars)

In [ ]:
from mapsy.data import Grid
contactspacegrid = Grid(scalars=scalars,cell=system.grid.cell)

Before creating a `ContactSpace` instance, we need to specify the type of boundary, choosing between a simple interface or a soft-sphere one. The following command generates a soft-sphere interface, with radii according to the UFF defintion, scaled by a factor `alpha` and with a softness of 1.

In [ ]:
from mapsy.boundaries import IonicBoundary
boundary = IonicBoundary(mode = 'muff', grid=contactspacegrid, alpha=1.12, softness=1.0, system = system, label='ionic')

For a simplified boundary centered on the system we could use (since the system is two-dimensional, this will generate a flat 2D interface)

In [ ]:
from mapsy.boundaries import SystemBoundary
boundary = SystemBoundary(mode = 'system', grid=contactspacegrid, distance = 3, spread = 1, system = system, label='system')

In [ ]:
boundary.update()

Given the boundary, we can create a `ContactSpace` by specifying the threshold on the modulus of the gradient that defines the contact space points. By default this threshold is set to 0.1, i.e., only points for which the modulus of the gradient of the boundary is larger than 0.1 (in internal units) will be included in the contact space points. 

In [ ]:
from mapsy.boundaries import ContactSpace
contactspace = ContactSpace(boundary, 0.1)